<a href="https://colab.research.google.com/github/Ayushverma135/Whisper-Hindi-ASR-model-IIT-Bombay-Intership/blob/main/Final_Submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU Check

In [ ]:
!nvidia-smi

Mon Apr 22 14:45:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Install the Whisper Library

In [ ]:
! pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.6 MB/s eta 0:00:00


# Load and Import the Model and Library

In [ ]:
import whisper

model = whisper.load_model("large") #769M Parameter

100%|██████████████████████████████████████| 2.88G/2.88G [00:17<00:00, 173MiB/s]


# Download Audio direct from the website

In [ ]:
!wget -O audio.mp3 http://www.moviesoundclips.net/movies1/batmanbegins/bats.mp3

--2024-04-20 17:17:28--  http://www.moviesoundclips.net/movies1/batmanbegins/bats.mp3
Resolving www.moviesoundclips.net (www.moviesoundclips.net)... 198.54.115.219
Connecting to www.moviesoundclips.net (www.moviesoundclips.net)|198.54.115.219|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52128 (51K) [audio/mpeg]
Saving to: ‘audio.mp3’

audio.mp3           100%[===================>]  50.91K  --.-KB/s    in 0.04s   

2024-04-20 17:17:28 (1.24 MB/s) - ‘audio.mp3’ saved [52128/52128]



# Mount the Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Display the Downloaded audio from the google drive

In [ ]:
from IPython.display import Audio, display

display(Audio('/content/drive/MyDrive/Audio/01-00004-02.mp3', autoplay=True))

# Read and Print mp3 Files

In [ ]:
import glob
directory = '/content/drive/MyDrive/ResultText4.txt'

# Define the pattern to search for MP3 files
pattern = '*.mp3'

# Use glob to find MP3 files
mp3_files = glob.glob(directory + '**/' + pattern, recursive=True)

# Print the paths of MP3 files
for mp3_file in mp3_files:
    result = model.transcribe(mp3_file,language='hi')
    filename = mp3_file.split('/')[-1].split('.')[0]
    print(filename,result["text"])

02-15237-01  पर पर्खण परमुख राणी देवी ने उठाए कई सवाल पर पर्खण परमुख राणी देवी ने बाल विकास परिज़ना कारियाले में भरस्टाचार पूरी तरा व्याप्त है


KeyboardInterrupt: 

In [ ]:
result = model.transcribe("/content/drive/MyDrive/Audio/01-00004-02.mp3",language='hi')
print(result["text"])

 वहीं राहुल कुमार बुगेश कुमार राजा कुमार अनुरख कुमार शमी कुमार रहेश कुमार मधिस कुमार शाहित गर्जेनों कार्यकार भी सामन से जन्मवक दिलपांडे के मौगरा से बिहार मुझवानी लिजर प्रेस्टूर्स


# Transcribe the Audio Files into Text and Write

In [ ]:
import glob

directory = '/content/drive/MyDrive/Audio/'

# Define the pattern to search for MP3 files
pattern = '*.mp3'

# Use glob to find MP3 files
mp3_files = glob.glob(directory + '**/' + pattern, recursive=True)

# Open a text file to write results
with open('/content/drive/MyDrive/ResultText4.txt', 'w') as file:
    # Iterate over each MP3 file
    for mp3_file in mp3_files:
        result = model.transcribe(mp3_file, language='hi')  # Assuming "model" is defined elsewhere
        filename = mp3_file.split('/')[-1].split('.')[0]
        file.write(filename + ': ' + result["text"] + '\n')


# Find the Word Error Rate(WER)

In [ ]:
import glob

def wer(reference, hypothesis):
    """
    Calculate Word Error Rate (WER) between reference and hypothesis texts.
    Args:
        reference (str): Reference text (ground truth).
        hypothesis (str): Hypothesis text (recognized/transcribed).
    Returns:
        float: Word Error Rate (WER).
    """
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()

    # Create a matrix to store edit distances between all pairs of reference and hypothesis words
    dp = [[0] * (len(hypothesis_words) + 1) for _ in range(len(reference_words) + 1)]

    # Initialize first row and first column of the matrix
    for i in range(len(reference_words) + 1):
        dp[i][0] = i
    for j in range(len(hypothesis_words) + 1):
        dp[0][j] = j

    # Fill the matrix
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(hypothesis_words) + 1):
            if reference_words[i - 1] == hypothesis_words[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j - 1], dp[i - 1][j], dp[i][j - 1]) + 1

    # Return the WER
    return dp[len(reference_words)][len(hypothesis_words)] / len(reference_words)


# Example usage with Hindi text:
with open('/content/drive/MyDrive/ResultTextWER.txt', 'w') as file:
  # filename = mp3_file.split('/')[-1].split('.')[0]

  # Read the ground truth text file
  hypothesis_file = f'/content/drive/MyDrive/ResultText4.txt'
  reference_file = f'/content/text.txt' # Change path accordingly
  with open(hypothesis_file, 'r') as f:
    hypothesis_text = f.read().split(':')[-1]
  with open(reference_file, 'r') as f:
    reference_text = f.read().split(':')[-1]
    # Calculate WER using asr-evaluation
    wer_score = wer(reference_text, hypothesis_text)

    # Write results to file
    file.write(mp3_file + ': ' + result["text"] + '\n')
    file.write('WER: ' + str(wer) + '\n\n')
reference_text = "मैं खुश हूँ"
hypothesis_text = "मैं बहुत खुश हूँ"
wer_score = wer(reference_text, hypothesis_text)
print("WER:", wer_score)


WER: 0.3333333333333333


# Compare and Print Transcribed File with Kathbath Dataset Reference File

In [ ]:
import re
def wer_from_files(reference_file, hypothesis_file):
    """
    Calculate Word Error Rate (WER) between texts from reference and hypothesis files.
    Args:
        reference_file (str): File path to the reference text file (ground truth).
        hypothesis_file (str): File path to the hypothesis text file (recognized/transcribed).
    Returns:
        float: Word Error Rate (WER).
    """
    # Read reference text from file
    with open(reference_file, 'r', encoding='utf-8') as f:
        # reference_text = f.read().strip().splitlines()
        reference_text = f.read()
        pattern = r'[\u0900-\u097F\s]+'

    # Find all matches of Hindi sentences
        hindi_sentences = re.findall(pattern, reference_text)
    print(hindi_sentences)
    # Join the matches to form the complete Hindi sentence
    # complete_hindi_sentence = ''.join(hindi_sentences)
    # print(complete_hindi_sentence)
    # Read hypothesis text from file
    with open(hypothesis_file, 'r', encoding='utf-8') as f:
        hypothesis_text = f.read().split(':')[-1]

    # Calculate total number of words in reference text
    total_words = sum(len(line.split()) for line in hindi_sentences)

    # Calculate total number of errors between reference and hypothesis texts
    total_errors = sum(wer(reference_line, hypothesis_line) * len(reference_line.split()) for reference_line, hypothesis_line in zip(hindi_sentences, hypothesis_text))

    # Calculate Word Error Rate (WER)
    wer_score = total_errors / total_words
    return wer_score

def wer(reference, hypothesis):
    """
    Calculate Word Error Rate (WER) between reference and hypothesis texts.
    Args:
        reference (str): Reference text (ground truth).
        hypothesis (str): Hypothesis text (recognized/transcribed).
    Returns:
        float: Word Error Rate (WER).
    """
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()

    # Create a matrix to store edit distances between all pairs of reference and hypothesis words
    dp = [[0] * (len(hypothesis_words) + 1) for _ in range(len(reference_words) + 1)]

    # Initialize first row and first column of the matrix
    for i in range(len(reference_words) + 1):
        dp[i][0] = i
    for j in range(len(hypothesis_words) + 1):
        dp[0][j] = j

    # Fill the matrix
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(hypothesis_words) + 1):
            if reference_words[i - 1] == hypothesis_words[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j - 1], dp[i - 1][j], dp[i][j - 1]) + 1

    # Return the WER
    return dp[len(reference_words)][len(hypothesis_words)] / len(reference_words)

# Example usage with file paths:
reference_file_path = '/content/drive/MyDrive/text.txt'
hypothesis_file_path = '/content/drive/MyDrive/ResultText4.txt'
wer_score = wer_from_files(reference_file_path, hypothesis_file_path)
print("WER:", wer_score)


[' वहीं राहुल कुमार मुकेश कुमार अरुण कुमार राजा कुमार अनुराग कुमार सनी कुमार रहीश कुमार मनीष कुमार सहित दर्जनों कार्यकर्ता भी शामिल थे धन्यवाद दिलीप पांडे जमुई अघहारा से बिहार मोबइल वाणी न्यूज़ एक्सप्रेस से\n', ' युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या पैसे खर्च करने की ज़रुरत नहीं है  दिन\n', ' क्यूँकि हमारी पुलिस तो फेसबुक और दुसरे सोशल मीडिया के माध्यमों पर अपराध करने वालों को पकड़ने में ही असमर्थ है\n', ' नाली जाम है जिससे निकालता गन्दा पानी सड़को पर जमा है जिसकी सफाई को लेकर किसी प्रकार की कयावत होती भी नहीं दिखती है\n', ' समस्या को सुना जायेगा और महिलाओं की समस्या महिलाओं समझ सकती है इस इस विचार को रखते हुए सरकार ने महिलाओं के प्र \n', ' विकास के लिए शराब बंदी एक बड़ा कदम है इस मुद्दे पर जदूय जि ज़िला अध्यक्ष शंकर चौधरी जदूय नेता\n', ' इस मध्य प्रदेश मोबाइल वाणी के सभी सुनने वाले श्रोताओं को और मेरे मित्रों को सभी को हार्दिक हार्दिक शुभकामनाएं धन्यवाद नमस्कार\n', ' स्तर गिरने का मुख्य कारण  संस्कृति और परम्पराओं का सबक सबक बच्चों को न पढ़ाना है दुर्भाग्य की बात है कि भारत मैं आज लो

In [ ]:
import re
text="01-00008-03 युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या पैसे खर्च करने की ज़रुरत नहीं है  दिन"
# Define the pattern to extract Hindi sentences
pattern = r'[\u0900-\u097F\s]+'

# Find all matches of Hindi sentences
hindi_sentences = re.findall(pattern, text)

# Join the matches to form the complete Hindi sentence
complete_hindi_sentence = ''.join(hindi_sentences)

print(complete_hindi_sentence)

 युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या पैसे खर्च करने की ज़रुरत नहीं है  दिन
